In [1]:
from __future__ import print_function

import numpy as np

import scipy.misc
import scipy.io

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.applications import vgg19
from keras.models import load_model, Model, Sequential
from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Lambda, Flatten, Dense
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Reshape
from keras.layers.merge import concatenate
import tensorflow as tf

# Set backend as tensorflow
from keras import backend as K
K.set_image_dim_ordering("tf")

from PIL import Image
import random
import os
import h5py

%matplotlib inline

Using TensorFlow backend.


In [2]:
BASE_DIR = "./ships-in-satellite-imagery/shipsnet/"
TRAIN_ROWS = .8
IMAGE_HEIGHT = 80
IMAGE_WIDTH = 80
COLOR_CHANNELS = 3

In [3]:
def readFromImages():
    file_list = os.listdir(BASE_DIR)
    file_list = random.sample(file_list, len(file_list))

    x_data = np.zeros((total_images, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS))
    y_data = np.zeros((total_images, 1))

    for f in range(total_images):
        filename = file_list[f]
        x_data[f] = plt.imread(BASE_DIR + filename, "r")
        y_data[f] = int(filename.split("_", 1)[0])

    h5File = h5py.File("data.h5", "w")
    h5File.create_dataset("x_data", data=x_data)
    h5File.create_dataset("y_data", data=y_data)
    h5File.close()
    
    return x_data, y_data

In [4]:
def readFromFile():
    h5File = h5py.File("data.h5")
    x_data = h5File["x_data"][:]
    y_data = h5File["y_data"][:]
            
    return x_data, y_data

In [5]:
x_data, y_data = readFromFile()
total_images = x_data.shape[0]

train_rows = int(total_images * TRAIN_ROWS)

x_train = x_data[:train_rows,:,:,:]
y_train = y_data[:train_rows,:]
x_test = x_data[train_rows:,:,:,:]
y_test = y_data[train_rows:,:]

In [15]:
input_image = Input(shape=(80, 80, 3), name="input_1")

x = Conv2D(32, (3, 3), name="conv_1", padding="same")(input_image)
x = BatchNormalization(name='norm_1')(x)
x = LeakyReLU(alpha=0.5, name="leaky_relu_1")(x)
x = MaxPooling2D(pool_size=(2, 2), name="max_pool_1")(x)

x = Conv2D(64, (3, 3), name="conv_2", padding="same")(x)
x = BatchNormalization(name='norm_2')(x)
x = LeakyReLU(alpha=0.5, name="leaky_relu_2")(x)

x = Conv2D(64, (3, 3), name="conv_3", padding="same")(x)
x = BatchNormalization(name='norm_3')(x)
x = LeakyReLU(alpha=0.5, name="leaky_relu_3")(x)
x = MaxPooling2D(pool_size=(2, 2), name="max_pool_2")(x)

x = Conv2D(128, (3, 3), name="conv_4", padding="same")(x)
x = BatchNormalization(name='norm_4')(x)
x = LeakyReLU(alpha=0.5, name="leaky_relu_4")(x)
x = MaxPooling2D(pool_size=(2, 2), name="max_pool_3")(x)

x = Conv2D(64, (3, 3), name="conv_5", padding="same")(x)
x = BatchNormalization(name='norm_5')(x)
x = LeakyReLU(alpha=0.1, name="leaky_relu_5")(x)
x = MaxPooling2D(pool_size=(2, 2), name="max_pool_4")(x)

x = Conv2D(32, (3, 3), name="conv_6", padding="same")(x)
x = BatchNormalization(name='norm_6')(x)
x = LeakyReLU(alpha=0.1, name="leaky_relu_6")(x)
x = MaxPooling2D(pool_size=(2, 2), name="max_pool_5")(x)

x = Conv2D(16, (3, 3), name="conv_7", padding="same")(x)
x = BatchNormalization(name='norm_7')(x)
x = LeakyReLU(alpha=0.1, name="leaky_relu_7")(x)
x = MaxPooling2D(pool_size=(2, 2), name="max_pool_6")(x)

x = Flatten(name="flatten_1")(x)
x = Dense(1, name="dense_1")(x)

model = Model(inputs=input_image, outputs=x)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 80, 80, 3)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 80, 80, 32)        896       
_________________________________________________________________
norm_1 (BatchNormalization)  (None, 80, 80, 32)        128       
_________________________________________________________________
leaky_relu_1 (LeakyReLU)     (None, 80, 80, 32)        0         
_________________________________________________________________
max_pool_1 (MaxPooling2D)    (None, 40, 40, 32)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 40, 40, 64)        18496     
_________________________________________________________________
norm_2 (BatchNormalization)  (None, 40, 40, 64)        256       
__________

In [16]:
# For a binary classification problem
model.compile(optimizer='adam',
              loss='mean_squared_logarithmic_error',
              metrics=['accuracy'])

In [ ]:
# Train the model, iterating on the data in batches of 32 samples
model.fit(x_train, y_train, epochs=4, batch_size=32)

Epoch 1/4
2240/2240 [==============================] - 176s 78ms/step - loss: 0.0512 - acc: 0.5375
Epoch 2/4
1152/2240 [==============>...............] - ETA: 1:21 - loss: 0.0214 - acc: 0.6962

In [21]:
model.evaluate(x_test, y_test)

560/560 [==============================] - 13s 23ms/step


[0.0087508548982441429, 0.61250000000000004]